# 操控 {class}`~torch.fx.Graph`

构建新 {class}`~torch.fx.Graph` 的一种方法是直接操控旧图。为了帮助实现这一点，可以简单地从符号跟踪中获取 {class}`~torch.fx.Graph` 并对其进行修改。例如，假设希望用 {func}`torch.mul` 调用替换 {func}`torch.add` 调用。

In [1]:
import torch
from torch import fx

# Sample module
class M(torch.nn.Module):
    def forward(self, x, y):
        return torch.add(x, y)

def transform(m: torch.nn.Module,
              tracer_class : type = fx.Tracer) -> torch.nn.Module:
    graph : fx.Graph = tracer_class().trace(m)
    # FX represents its Graph as an ordered list of
    # nodes, so we can iterate through them.
    for node in graph.nodes:
        # Checks if we're calling a function (i.e:
        # torch.add)
        if node.op == 'call_function':
            # The target attribute is the function
            # that call_function calls.
            if node.target == torch.add:
                node.target = torch.mul

    graph.lint() # Does some checks to make sure the
                 # Graph is well-formed.

    return fx.GraphModule(m, graph)

还可以进行更复杂的 {class}`~torch.fx.Graph` 重写，比如删除或追加节点。为了帮助完成这些变换，FX 提供了变换 {class}`~torch.fx.Graph` 的实用函数。下面是使用这些 API 附加 {func}`torch.relu` 调用的示例。

```python
# Specifies the insertion point. Any nodes added to the
# Graph within this scope will be inserted after `node`
with traced.graph.inserting_after(node):
    # Insert a new `call_function` node calling `torch.relu`
    new_node = traced.graph.call_function(
        torch.relu, args=(node,))

    # We want all places that used the value of `node` to
    # now use that value after the `relu` call we've added.
    # We use the `replace_all_uses_with` API to do this.
    node.replace_all_uses_with(new_node)
```

对于仅由替换组成的简单变换，还可以使用 {mod}`torch.fx.subgraph_rewriter`。

## {func}`~torch.fx.replace_pattern` 重写子图

FX 在直接 {func}`~torch.fx.Graph` 操作的基础上还提供了另一个自动化级别。{func}`~torch.fx.replace_pattern` API 本质上是编辑 {func}`~torch.fx.Graph` 的“查找/替换”工具。它允许您指定 `pattern` 和 `replacement`，它将跟踪这些函数，在 `pattern` graph 中查找运算组的实例，并用 `replacement` graph 的副本替换这些实例。这有助于极大地自动化繁琐的 graph 操作代码，随着变换变得更加复杂，这些代码可能会变得笨拙。

## Proxy/Retracing

另一种操作 {class}`~torch.fx.Graph` 的方法是重用符号跟踪中使用的 {class}`~torch.fx.Proxy` 机制。例如，假设想要编写一个变换，将 PyTorch 函数分解为更小的运算。它将把每个 `F.relu(x)` 调用变换为 `(x > 0) * x`。一种可能是执行必要的 graph 重写，在 `F.relu` 之后插入比较和乘法，然后清理原来的 `F.relu`。但是，可以通过使用 {class}`~torch.fx.Proxy` 对象自动地将操作记录到 {class}`~torch.fx.Graph` 中来自动化这个过程。

要使用此方法，将希望插入的操作编写为常规 PyTorch 代码，并使用 {class}`~torch.fx.Proxy` 对象作为参数调用该代码。这些代理对象将捕获对它们执行的操作，并将它们附加到 {class}`~torch.fx.Graph` 中。

In [ ]:
# Note that this decomposition rule can be read as regular Python
def relu_decomposition(x):
    return (x > 0) * x

decomposition_rules = {}
decomposition_rules[F.relu] = relu_decomposition

def decompose(model: torch.nn.Module,
              tracer_class : type = fx.Tracer) -> torch.nn.Module:
    """
    Decompose `model` into smaller constituent operations.
    Currently,this only supports decomposing ReLU into its
    mathematical definition: (x > 0) * x
    """
    graph : fx.Graph = tracer_class().trace(model)
    new_graph = fx.Graph()
    env = {}
    tracer = torch.fx.proxy.GraphAppendingTracer(graph)
    for node in graph.nodes:
        if node.op == 'call_function' and node.target in decomposition_rules:
            # By wrapping the arguments with proxies,
            # we can dispatch to the appropriate
            # decomposition rule and implicitly add it
            # to the Graph by symbolically tracing it.
            proxy_args = [
                fx.Proxy(env[x.name], tracer) if isinstance(x, fx.Node) else x for x in node.args]
            output_proxy = decomposition_rules[node.target](*proxy_args)

            # Operations on `Proxy` always yield new `Proxy`s, and the
            # return value of our decomposition rule is no exception.
            # We need to extract the underlying `Node` from the `Proxy`
            # to use it in subsequent iterations of this transform.
            new_node = output_proxy.node
            env[node.name] = new_node
        else:
            # Default case: we don't have a decomposition rule for this
            # node, so just copy the node over into the new graph.
            new_node = new_graph.node_copy(node, lambda x: env[x.name])
            env[node.name] = new_node
    return fx.GraphModule(model, new_graph)

除了避免显式的 {class}`~torch.fx.Graph` 操作之外，使用 {class}`~torch.fx.Proxy` 还允许将重写规则指定为原生 Python 代码。对于需要大量重写规则的变换（如 vmap 或 grad），这通常可以提高规则的可读性和可维护性。注意，在调用 {class}`~torch.fx.Proxy` 时，还传递了指向底层变量 graph 的跟踪器。如果 graph 中的操作是 n-ary 的（例如 add 是二进制算子），那么调用 {class}`~torch.fx.Proxy` 不会创建 graph 跟踪器的多个实例，这会导致意外的运行时错误。推荐这种使用 {class}`~torch.fx.Proxy` 的方法，特别是当底层算子不能被安全地假定为一元的时候。